In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0971987247467041
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 14.269554169373405
200000 24.64688660422644
300000 29.55197784605893
400000 31.363894512406063
500000 29.929829971035808
600000 31.049086628894973
700000 30.258997545460048
800000 29.680685169185047
900000 29.100734416891328
1000000 31.03913600957883
1100000 29.01717451756828
1200000 14.01281165122678
1300000 24.58030788105014
1400000 28.48097370114231
1500000 29.94473104277586
1600000 34.65747269548895
1700000 28.814090595473825
1800000 30.683306997898853
1900000 29.061013426887282
2000000 28.822317750121158
2100000 29.145266779185455
2200000 29.4926711756883
2300000 29.741991040337762
2400000 7.718632300569232
2500000 31.359849523306238
2600000 36.45136873145522
2700000 36.12299123049128
2800000 35.73713142269963
2900000 36.30155278917783
3000000 34.68399

29500000 17.23108742896673
29600000 26.820658298477625
29700000 28.0178268703982
29800000 28.126395746486043
29900000 29.27690752487796
30000000 27.934685255727345
30100000 31.048292215127073
30200000 37.29781743364679
30300000 32.47418879543342
30400000 12.690171007795096
30500000 29.8732389082144
30600000 29.886003681706832
30700000 30.075033297528414
30800000 26.4023791344452
30900000 15.376293692243642
31000000 29.751539757379124
31100000 29.734266298691743
31200000 29.39050581179701
31300000 28.77332705108965
31400000 28.631653369570856
31500000 28.292968397194535
31600000 28.651746527050435
31700000 27.463548784896265
31800000 8.009324342639601
31900000 27.906685135674476
32000000 27.193919357943805
32100000 29.012154417181797
32200000 28.554690034759886
32300000 29.53699961823787
32400000 28.900066599238237
32500000 30.715708010503494
32600000 29.448277973038127
32700000 19.670362618198514
32800000 22.2999047691989
32900000 29.831332027446688
33000000 30.395137170180973
33100000

59300000 11.836986578699614
59400000 25.074006423400625
59500000 25.868936901886993
59600000 25.740851651086714
59700000 24.5074783583458
59800000 24.242457757868127
59900000 24.257532219218746
60000000 13.838403703565344
60100000 17.97752378657738
60200000 24.998388336314516
60300000 24.19530682904905
60400000 24.719936933001474
60500000 25.41098041428112
60600000 25.881343742630477
60700000 21.732200418448095
60800000 21.945247589454656
60900000 29.289385436350194
61000000 27.9848190819026
61100000 27.08067355286525
61200000 26.81109799226806
61300000 27.214860143894384
61400000 27.10923389555926
61500000 27.40315163747745
61600000 26.281898115538507
61700000 10.3600892884843
61800000 25.904962129068682
61900000 25.385638090020464
62000000 24.80975241557673
62100000 24.455864508702746
62200000 23.899170563149614
62300000 24.975998697413583
62400000 6.241303941596001
62500000 24.593396544222507
62600000 23.462340401940025
62700000 23.811503452440594
62800000 23.76596530006648
62900000

89100000 23.3430250187301
89200000 24.15850951195228
89300000 22.913381832419866
89400000 24.479726540866093
89500000 24.42624023277361
89600000 23.885291763137495
89700000 24.229909546841398
89800000 23.597422973270458
89900000 23.046710649623666
90000000 22.129977128319744
90100000 12.276634920519001
90200000 17.87861747253093
90300000 20.963292920076423
90400000 22.75426649102953
90500000 21.460526123862852
90600000 9.13558237325034
90700000 19.239748619783878
90800000 21.35215431856284
90900000 21.18527592334393
91000000 21.290035377905337
91100000 10.736089014553363
91200000 20.455065772513965
91300000 22.68391619848652
91400000 22.484392311687973
91500000 23.341906595779125
91600000 22.96678565483122
91700000 25.19324456174772
91800000 31.05407724831417
91900000 25.699205315063875
92000000 24.18020048158568
92100000 24.47433127146471
92200000 23.728357351389654
92300000 22.544861641086648
92400000 22.239534187120725
92500000 7.785019854641666
92600000 21.56436633908831
92700000 2

118200000 20.248109627328297
118300000 19.23036360999217
118400000 19.50680933883852
118500000 17.0861554396475
118600000 9.551699430264204
118700000 20.008125952219668
118800000 13.31853502049014
118900000 11.31192709964534
119000000 19.069632833698762
119100000 12.247810563232036
119200000 15.649522081693942
119300000 19.536480821683792
119400000 19.01189168164197
119500000 19.653271050797
119600000 19.79937386555951
119700000 20.190728336033118
119800000 20.84294176195028
119900000 21.576617607673416
120000000 21.1601834006724
120100000 22.472047432688864
120200000 21.510553278906563
120300000 21.165560906117996
120400000 20.041227853679594
120500000 20.007692007248416
120600000 20.2363233145429
120700000 19.59588256824334
120800000 19.476797255991745
120900000 12.669585859654909
121000000 14.067769288033332
121100000 18.966980650829925
121200000 5.650155284340803
121300000 19.62734209509938
121400000 17.317713620348446
121500000 10.83372070655416
121600000 19.787555575848973
121700

...

261600000 5.812344234539083
261700000 10.008779855347145
261800000 9.609822890390458
261900000 9.580757697184342
262000000 9.630156191888725
262100000 9.084186302330963
262200000 9.268651875053278
262300000 7.431133156807023
262400000 6.152948026467971
262500000 11.442849019725104
262600000 9.745288645373773
262700000 9.464349509773113
262800000 9.530671191730367
262900000 9.556082708088201
263000000 9.99572678354695
263100000 8.13987634643215
263200000 5.776187520343888
263300000 10.068777521583876
263400000 9.97267157415754
263500000 10.216242410392756
263600000 10.237520295117102
263700000 9.93525077630219
263800000 10.122498398002238
263900000 3.8563689446175755
264000000 10.57675205819116
264100000 9.849402131771793
264200000 9.449276778667207
264300000 9.32755592534446
264400000 9.284793464898547
264500000 9.136477824106775
264600000 8.897962073283008
264700000 3.573711131031269
264800000 8.303266505824837
264900000 9.048722505855276
265000000 9.052228172024096
265100000 9.776671

290900000 7.757521737930651
291000000 7.753725214087954
291100000 7.638342309427681
291200000 7.848768359488147
291300000 7.76183852118494
291400000 7.821697052762276
291500000 8.000181408217124
291600000 2.8656418403302704
291700000 8.135236957163691
291800000 7.774570223351186
291900000 4.253030959137675
292000000 8.142335064275843
292100000 8.642051136830737
292200000 7.5558171792578115
292300000 7.763905426647979
292400000 7.5357943870010615
292500000 7.451961647408906
292600000 7.5838860908566295
292700000 7.549852772189904
292800000 7.296496309154868
292900000 4.781571265635431
293000000 4.685998713659265
293100000 7.196296923422185
293200000 7.616030493849516
293300000 7.355493864023095
293400000 7.648524484174467
293500000 7.710766357529342
293600000 7.73457282333517
293700000 7.371427501050559
293800000 7.7151762578584835
293900000 7.1908618771004
294000000 3.6054579031724594
294100000 7.470243586752464
294200000 6.111052302596052
294300000 5.06160176714948
294400000 7.6240053

319800000 2.4870941891276486
319900000 2.631669116343431
320000000 2.7527291250444157
320100000 2.4923565538406245
320200000 2.604335750834735
320300000 2.4560729740546647
320400000 2.360599428144272
320500000 2.4682169250404056
320600000 2.3919298202383894
320700000 2.1143401734416436
320800000 2.3873640073520823
320900000 2.143696175969201
321000000 1.7405756549546068
321100000 1.3255668491560875
321200000 1.2880924404880114
321300000 1.2490531636379374
321400000 1.544058402674055
321500000 2.4494647930752764
321600000 2.6762805295432295
321700000 2.191010954057419
321800000 1.9673107884861072
321900000 2.4219965770994816
322000000 2.374239171675028
322100000 2.315032207410441
322200000 2.3369518522559045
322300000 2.803702684008712
322400000 2.5095789140239484
322500000 2.4613006217779683
322600000 2.4019910955041945
322700000 2.3634277148375227
322800000 2.198482485653224
322900000 2.1300956310534955
323000000 2.091234431469336
323100000 2.5151320790293097
323200000 2.3018782089455

348400000 1.4396838311810314
348500000 1.3969719750045837
348600000 1.3429212645490394
348700000 1.4382410515521427
348800000 1.1775982148517032
348900000 1.261473222104666
349000000 1.4571140403173406
349100000 1.444911493870838
349200000 1.0164321223691795
349300000 0.7115725281564295
349400000 0.7680448472879112
349500000 0.7626214729252717
349600000 0.9254719283863801
349700000 1.1455084396373312
349800000 1.4486910591116802
349900000 1.2383572038880588
350000000 1.3041104419628362
350100000 1.3448863920096674
350200000 1.2258953216364263
350300000 1.2481052549171474
350400000 1.4987852498932237
350500000 1.3713939832122344
350600000 1.4807877925512922
350700000 1.3633802185183321
350800000 1.4113338678775826
350900000 1.2765189915561377
351000000 1.2549488238397606
351100000 1.2485286795125128
351200000 1.2912934207760434
351300000 1.261459080827882
351400000 1.2888034623224756
351500000 1.0941586443590106
351600000 0.8907518567069048
351700000 0.7638510542292735
351800000 0.72007

376100000 0.05682026461293631
376200000 0.04393431124550965
376300000 0.04828898520476819
376400000 0.054120364792568154
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.2539578968619274
376600000 0.22798937023105423
376700000 0.17592574599937136
376800000 0.1646549991264568
376900000 0.16030595335025588
377000000 0.1863632725144691
377100000 0.15719082650763658
377200000 0.13121694132955075
377300000 0.1218540823767821
377400000 0.13032822446568013
377500000 0.1836739055379768
377600000 0.12035401673596567
377700000 0.120231996289087
377800000 0.12048374675587219
377900000 0.11716747128838634
378000000 0.11934854397552279
378100000 0.1142341488698125
378200000 0.1209753385875192
378300000 0.11767702729924719
378400000 0.13334410385270118
378500000 0.14497407663297787
378600000 0.1349941451438513
378700000 0.12738441044232646
378800000 0.10681008193186786
378900000 0.027133641633058255
379000000 0.030552327587968112
379100000 0.027012348404178356
379200